In [1]:
import linear_dag as ld
from linear_dag.brick_graph_py import BrickGraphPy
import numpy as np
import os
import networkx as nx
from linear_dag.linear_arg_inference import *
from scipy.sparse import csc_matrix, csr_matrix, eye, triu, tril
from scipy.sparse.linalg import inv
from miscellaneous import say_done, af_equal

In [2]:
path = "/Users/loconnor/Dropbox/linearARG/linearg_shared/data/1kg_ALL_chr22_47876022_49355984_MAF_0.01_RF_0.01_T_8.genos"
genotypes, _, _ = ld.genotype.load_genotypes(path)

kept_variants: (16233,)


In [3]:
geno_small = genotypes[:, :1000]
ac = np.diff(geno_small.indptr)
geno_small = geno_small[:, ac > 1]

In [4]:
linarg = ld.LinearARG.from_genotypes(geno_small, recombination_method="after", brick_graph_method="new")


0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
8
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
18
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
22
0
0
0
0
0
23
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

In [4]:
BG = ld.BrickGraph.from_genotypes(geno_small)

In [4]:
linarg = ld.LinearARG.from_genotypes(geno_small, recombination_method="before")
say_done()
af_equal(geno_small[:500,:], linarg[:500,:])

before


np.False_

In [7]:
af_equal(geno_small[:495,:], linarg[:495,:])


np.False_

In [70]:
geno_small.nnz, csr_matrix(geno_small).nnz

(1412742, 1412742)

In [33]:
brick_graph_closure = ld.BrickGraph.from_genotypes(geno_small).to_csr()


In [34]:
brick_graph_closure = remove_undirected_edges(brick_graph_closure)

In [81]:
from scipy.sparse import block_diag
brick_graph_closure_samples = add_samples_to_brick_graph_closure(csc_matrix((0, brick_graph_closure.shape[0])), brick_graph_closure)
brick_graph_closure_samples = csr_matrix(block_diag([brick_graph_closure, csr_matrix((1,1))]))
setdiag(brick_graph_closure_samples, 0)
brick_graph_closure_samples.eliminate_zeros()

In [82]:
brick_graph = csr_matrix(closure_transitive_reduction(brick_graph_closure_samples))

In [51]:
csc_matrix((1, brick_graph_closure.shape[0]))

<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 0 stored elements and shape (1, 9664)>

In [68]:
A = csc_matrix(np.ones((1,2)))
B = csr_matrix(triu(csr_matrix(np.ones((2,2)))))
C = add_samples_to_brick_graph_closure(A, B)
setdiag(C, 0)
C.eliminate_zeros()
closure_transitive_reduction(C)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2 stored elements and shape (3, 3)>

In [69]:
geno_small

<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 1412742 stored elements and shape (500, 9664)>

In [11]:
setdiag(brick_graph_closure,0)
brick_graph_closure.eliminate_zeros()

In [12]:
brick_graph = csr_matrix(closure_transitive_reduction(brick_graph_closure))


In [16]:
A = brick_graph_closure_samples - (brick_graph_closure_samples@brick_graph_closure_samples > 0)

In [18]:
A.eliminate_zeros()

In [20]:
e = np.where(A.data != 1)[0]
e[:10]

array([275615, 276271, 276273, 276274, 276276, 276278, 276280, 276285,
       276287, 276289])

In [23]:
len(A.data), np.sum(A.data != 1)

(318074, np.int64(9539))

In [2]:
%%time
path = "/Users/loconnor/Dropbox/linearARG/linearg_shared/data/1kg_ALL_chr22_47876022_49355984_MAF_0.01_RF_0.01_T_8.genos"
output_file_prefix = "/Users/loconnor/Dropbox/linearARG/linearg_shared/data/linarg/1kg_ALL_chr22_47876022_49355984_MAF_0.01_RF_0.01_T_8"
linarg_old, genotypes = ld.run_linarg_workflow(
    path, output_file_prefix, flip_minor_alleles=True, recombination_method="", make_triangular=False, brick_graph_method="new"
)

kept_variants: (16233,)
file_name: /Users/loconnor/Dropbox/linearARG/linearg_shared/data/1kg_ALL_chr22_47876022_49355984_MAF_0.01_RF_0.01_T_8.genos,num_samples: 5008, num_variants: 16233, nnz_X: 7837269, nnz_A: : 5296252, nnz_ratio: 1.4797764532352313, runtime: 82.65161395072937
CPU times: user 1min 19s, sys: 1.5 s, total: 1min 21s
Wall time: 1min 22s


In [13]:
%%time
path = "/Users/loconnor/Dropbox/linearARG/linearg_shared/data/1kg_ALL_chr22_47876022_49355984_MAF_0.01_RF_0.01_T_8.genos"
output_file_prefix = "/Users/loconnor/Dropbox/linearARG/linearg_shared/data/linarg/1kg_ALL_chr22_47876022_49355984_MAF_0.01_RF_0.01_T_8"
linarg_old, genotypes = ld.run_linarg_workflow(
    path, output_file_prefix, flip_minor_alleles=True, recombination_method="", make_triangular=False, brick_graph_method="new"
)

kept_variants: (16233,)
file_name: /Users/loconnor/Dropbox/linearARG/linearg_shared/data/1kg_ALL_chr22_47876022_49355984_MAF_0.01_RF_0.01_T_8.genos,num_samples: 5008, num_variants: 16233, nnz_X: 7837269, nnz_A: : 4702138, nnz_ratio: 1.6667458505046002, runtime: 46.64146709442139
CPU times: user 43.5 s, sys: 3.55 s, total: 47 s
Wall time: 46.6 s


In [3]:
geno_small = genotypes[:500, :]
ac = np.diff(geno_small.indptr)
geno_small = geno_small[:, ac > 1]
linarg = ld.LinearARG.from_genotypes(geno_small, recombination_method="before")

before


ValueError: Input graph was not transitive

In [ ]:
say_done()
af_equal(geno_small[:500,:], linarg[:500,:])

In [23]:
brick_graph_closure = ld.BrickGraph.from_genotypes(geno_small).to_csr()
# brick_graph_closure = infer_brick_graph_using_containment(geno_small, 1)

In [24]:
brick_graph_closure = remove_undirected_edges(brick_graph_closure)
setdiag(brick_graph_closure, 0)
brick_graph_closure.eliminate_zeros()

In [25]:
result = brick_graph_closure - (brick_graph_closure @ brick_graph_closure > 0)
result.eliminate_zeros()

In [26]:
np.mean(result.data == 1)

np.float64(1.0)

In [7]:
linarg = linarg.make_triangular()
v = np.ones(linarg.shape[0])
np.all(v @ linarg == v @ geno_small)

ValueError: dimension mismatch

In [9]:
ac = np.diff(genotypes.indptr)
np.sum(ac > 50)

10962

In [8]:
n,m = linarg.shape
geno_ok = []
for i in range(n):
    v = np.zeros(n)
    v[i] = 1
    ok = np.all(v @ geno_small == v @ linarg)
    geno_ok.append(ok)

say_done()
np.where(np.array(geno_ok) != 1)

(array([499]),)

In [17]:
genotypes[5004,:]

<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 1379 stored elements and shape (1, 16233)>

In [18]:
v @ linarg

array([[0., 0., 0., ..., 0., 0., 0.]])

In [6]:
# linarg.A = csr_matrix(linarg.A.transpose())

In [7]:
# linarg = linarg.make_triangular()

In [8]:
print(v @ linarg)
print(v @ genotypes)

[[2000. 2464. 2420. ...   84.  668.  837.]]
[2000. 2464. 2420. ...   84.  668.  837.]


In [9]:
acl = v @ linarg
acg = v @ genotypes
print(np.where(acl != acg))

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [10]:
acl[0,156], acg[156]

(1351.0, 1352.0)

In [3]:
ploidy = 1
brick_graph_closure = infer_brick_graph_using_containment(genotypes, ploidy)
brick_graph_closure = remove_undirected_edges(brick_graph_closure)

In [4]:
setdiag(brick_graph_closure, 0)
brick_graph_closure.eliminate_zeros()

In [41]:
# # Brick graph including sample nodes
brick_graph_closure = add_samples_to_brick_graph_closure(genotypes, brick_graph_closure)
setdiag(brick_graph_closure, 0)
brick_graph_closure.eliminate_zeros()

In [42]:
brick_graph_closure

<Compressed Sparse Row sparse matrix of dtype 'int32'
	with 7952849 stored elements and shape (21241, 21241)>

In [28]:
# brick_graph = csr_matrix(closure_transitive_reduction(brick_graph_closure).transpose())
# setdiag(brick_graph, 0)
# brick_graph.eliminate_zeros()
# brick_graph.sort_indices()

In [29]:
# # Find recombinations
# trio_list = ld.Trios(brick_graph.nnz)
# trio_list.convert_matrix(brick_graph.indices, brick_graph.indptr)
# trio_list.find_recombinations()
# edges = trio_list.fill_edgelist()

In [5]:
from scipy.sparse import coo_matrix
M = coo_matrix(brick_graph_closure)
edges = list(zip(M.col, M.row))

In [6]:
# Linearize
import networkx as nx
G = nx.DiGraph()
num_nodes = 1 + np.max([max(i, j) for i, j in edges])
G.add_nodes_from(np.arange(num_nodes))
G.add_edges_from(edges)

G = nx.transitive_reduction(G)
# path_sums = ld.one_summed.compute_path_sums(G)

In [7]:
for e in G.edges:
    G.edges[e]['weight'] = 1

In [8]:
from one_summed import ActiveSet
def construct_1_summed_DAG_simple(G: nx.DiGraph) -> nx.DiGraph:
    from collections import defaultdict
    G_prime = G.copy()
    sorted_nodes = reversed(list(nx.topological_sort(G)))
    topological_ordering = {node: index for index, node in enumerate(sorted_nodes)}

    for i in topological_ordering:
        L = ActiveSet(i, -topological_ordering[i])
        path_sum = defaultdict(int)
        path_sum[i] = 1

        while L:
            j = L.pop()
            for k in G_prime.successors(j):
                assert k != j
                if k not in L:
                    L.push(k, -topological_ordering[k])
                path_sum[k] += G_prime[j][k]["weight"]

            if path_sum[j] == 1:
                continue
                
            if not G_prime.has_edge(i, j):
                G_prime.add_edge(i, j, weight=0)

            G_prime[i][j]["weight"] += 1 - path_sum[j]

    return G_prime

In [21]:
def add_samples_to_initial_brick_graph(genotypes: csc_matrix, brick_graph: nx.DiGraph) -> nx.DiGraph:
    n, m = genotypes.shape
    assert np.all(genotypes.data == 1), "this doesn't work with unphased data"
    
    result = G.copy()
    for row in range(n):
        row_data_indices = range(genotypes.indptr[row], genotypes.indptr[row + 1])
        ancestors = set(genotypes.indices[row_data_indices])
        parents = []
        for ancestor in ancestors:
            ancestor_parents = set(G.predecessors(ancestor))
            if ancestor_parents.isdisjoint(ancestors):
                parents.append(ancestor)
        result.add_weighted_edges_from([(parent, row+m, 1) for parent in parents])
        
    return result

In [9]:
Gp = construct_1_summed_DAG_simple(G.copy())

In [10]:
linear_arg_adjacency_matrix = csr_matrix(nx.to_scipy_sparse_array(Gp, format="csc").transpose())

In [13]:
linear_arg_adjacency_matrix_samples_added_after = add_samples_to_linear_arg(genotypes, linear_arg_adjacency_matrix)

In [14]:
n, m = linarg_old.shape
linarg_manual = ld.LinearARG(linear_arg_adjacency_matrix_samples_added_after, np.arange(n), np.arange(n, n+m), np.zeros(m))
say_done()
af_equal(genotypes, linarg_manual)

True

In [36]:
G_before = add_samples_to_initial_brick_graph(genotypes, G)
Gp_before = construct_1_summed_DAG_simple(G_before)

In [37]:
linear_arg_adjacency_matrix = csr_matrix(nx.to_scipy_sparse_array(Gp_before, format="csc").transpose())

In [44]:
n, m = linarg_old.shape
linear_arg_adjacency_matrix.eliminate_zeros()
linarg_manual = ld.LinearARG(linear_arg_adjacency_matrix, np.arange(m,n+m), np.arange(m), np.zeros(m))
say_done()
af_equal(genotypes, linarg_manual)

False

In [39]:
linarg_manual = linarg_manual.make_triangular()
v = np.ones(n)
v @ linarg_manual, v @ genotypes

(array([[1799., 1101., 1259., ...,    0.,    0.,    0.]]),
 array([2000., 2464., 2420., ...,   84.,  668.,  837.]))

In [35]:
np.unique(v @ linarg_manual - v @ genotypes)

array([-2503., -2501., -2500., ...,  2627.,  2628.,  2748.])

In [43]:
np.mean(linear_arg_adjacency_matrix_samples_added_after.data == 0)

0.0

In [ ]:
from scipy.sparse import coo_matrix
edges = coo_matrix(brick_graph)
edges = zip(edges.row, edges.col)

In [ ]:
import networkx as nx
from linear_dag.one_summed import construct_1_summed_DAG_slow
G = nx.DiGraph()
G.add_edges_from(edges)

In [3]:
nx.is_directed_acyclic_graph(G)

NameError: name 'nx' is not defined

In [4]:
for e in G.edges:
    G.edges[e]['weight'] = 1
Gp = construct_1_summed_DAG_slow(G)

NameError: name 'G' is not defined

In [5]:
for e in nx.selfloop_edges(G):
    G.remove_edge(*e)

NameError: name 'nx' is not defined

In [6]:
Gp = construct_1_summed_DAG_slow(G)

NameError: name 'construct_1_summed_DAG_slow' is not defined

In [7]:
edges = list(G.edges())
for i,j in edges:
    if not G.has_edge(j,i) or not G.has_edge(i,j):
        continue
    G.remove_edge(max(i,j), min(i,j))

NameError: name 'G' is not defined

In [8]:
Gp = construct_1_summed_DAG_slow(G)

NameError: name 'construct_1_summed_DAG_slow' is not defined

In [9]:
nx.is_directed_acyclic_graph(G)

NameError: name 'nx' is not defined

In [10]:
edges = trio_list.fill_edgelist()
np.asarray(edges)

NameError: name 'trio_list' is not defined

In [11]:
num_nodes = np.max(edges) + 1
new_brick_graph = csr_matrix(
            (np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), shape=(num_nodes, num_nodes))

NameError: name 'np' is not defined

In [12]:
from linear_dag.solve import topological_sort
from linear_dag.linear_arg_inference import setdiag

In [13]:
new_brick_graph.setdiag(0)
order = topological_sort(new_brick_graph)

NameError: name 'new_brick_graph' is not defined

In [ ]:
edges = trio_list.fill_edgelist()

In [ ]:
linear_arg_adjacency_matrix = csr_matrix(nx.to_scipy_sparse_array(G, format="csr"))

In [ ]:
brick_graph_closure.diagonal()

In [ ]:
brick_graph_closure.setdiag(np.zeros(brick_graph_closure.shape[0]), k=0)
brick_graph_closure.diagonal()


In [ ]:
brick_graph_closure.setdiag(np.zeros(brick_graph_closure.shape[0]), k=0)
brick_graph_closure.diagonal()


In [ ]:
brick_graph_closure.shape

In [ ]:
len(brick_graph_closure.indptr)

In [ ]:
max(brick_graph_closure.indices)

In [ ]:
np.mean(brick_graph_closure.diagonal())

In [ ]:
v = np.ones(linarg_old.shape[0])
v @ linarg_old, v @ genotypes

In [ ]:
linarg = ld.LinearARG.from_genotypes(genotypes, recombination_method="after", brick_graph_method="old", make_triangular=True)

In [ ]:
np.mean(linarg.data == 1)

In [ ]:
linarg.nnz

In [ ]:
linarg = linarg_old.unweight()
linarg = linarg_old.find_recombinations()
linarg.nnz

In [14]:
%%time
path = "/Users/loconnor/Dropbox/linearARG/linearg_shared/data/1kg_ALL_chr22_47876022_49355984_MAF_0.01_RF_0.01_T_8.genos"
output_file_prefix = None # "/Volumes/T7/data/test/1kg_chr22_47876022_49355984"
linarg_old, genotypes = ld.run_linarg_workflow(
    path, output_file_prefix, flip_minor_alleles=True, recombination_method="after", make_triangular=False, brick_graph_method="old"
)

NameError: name 'ld' is not defined

In [15]:
linarg_old = linarg_old.unweight()
linarg_old = linarg_old.find_recombinations()
linarg_old.nnz

NameError: name 'linarg_old' is not defined

In [16]:
genotypes.nnz / linarg_old.nnz

NameError: name 'genotypes' is not defined

In [17]:
%%time
from linear_dag.brick_graph import BrickGraph
bg = BrickGraph.from_genotypes(genotypes)

NameError: name 'genotypes' is not defined

In [18]:
edges = bg.edge_list()
len(edges)

NameError: name 'bg' is not defined

In [19]:
linarg_new = ld.LinearARG.from_genotypes(genotypes, brick_graph_method="new_cython")

NameError: name 'ld' is not defined

In [20]:
n = genotypes.shape[1]
rows, cols = zip(*edges)
result = csr_matrix((np.ones(len(edges)), (rows, cols)), (n,n))

NameError: name 'genotypes' is not defined

In [21]:
%%time
path = "/Users/loconnor/Dropbox/linearARG/linearg_shared/data/1kg_ALL_chr22_47876022_49355984_MAF_0.01_RF_0.01_T_8.genos"
output_file_prefix = None # "/Volumes/T7/data/test/1kg_chr22_47876022_49355984"
linarg_old, genotypes = ld.run_linarg_workflow(
    path, output_file_prefix, flip_minor_alleles=True, recombination_method="after_linearizing", make_triangular=False, brick_graph_method="new"
)

NameError: name 'ld' is not defined

In [22]:
linarg_old = linarg_old.unweight()
linarg_old = linarg_old.find_recombinations()

NameError: name 'linarg_old' is not defined

In [23]:
genotypes.nnz / linarg_old.nnz

NameError: name 'genotypes' is not defined

In [24]:
# %%time
# linarg_reorder = ld.LinearARG.from_genotypes(genotypes, brick_graph_method="old", recombine_before_linearize=True)

In [25]:
ploidy = 1
brick_graph_closure_init = infer_brick_graph_using_containment(genotypes, ploidy)

NameError: name 'infer_brick_graph_using_containment' is not defined

In [26]:
brick_graph_closure = brick_graph_closure_init.copy()

NameError: name 'brick_graph_closure_init' is not defined

In [27]:
brick_graph_closure = remove_undirected_edges(brick_graph_closure)

NameError: name 'remove_undirected_edges' is not defined

In [28]:
brick_graph_closure = add_samples_to_brick_graph(brick_graph_closure, genotypes)

NameError: name 'add_samples_to_brick_graph' is not defined

In [29]:
brick_graph_closure.setdiag(0) # necessary?
brick_graph_closure.eliminate_zeros()

NameError: name 'brick_graph_closure' is not defined

In [30]:
brick_graph = closure_transitive_reduction(brick_graph_closure)

NameError: name 'closure_transitive_reduction' is not defined

In [31]:
# _ = os.system("say done")

In [32]:
brick_graph, brick_graph_closure, genotypes

NameError: name 'brick_graph' is not defined

In [33]:
trio_list = ld.Trios(brick_graph.nnz)
trio_list.convert_matrix(brick_graph.indices, brick_graph.indptr)
trio_list.find_recombinations()

NameError: name 'ld' is not defined

In [34]:
edges = trio_list.fill_edgelist()
num_nodes = np.max(edges) + 1
new_brick_graph = csr_matrix((np.ones(edges.shape[0]), (edges[:, 1], edges[:, 0])), shape=(num_nodes, num_nodes))

NameError: name 'trio_list' is not defined

In [35]:
new_brick_graph.setdiag(0)
new_brick_graph.eliminate_zeros()
new_brick_graph.sort_indices()

NameError: name 'new_brick_graph' is not defined

In [36]:
brick_graph_closure = transitive_closure(new_brick_graph)

NameError: name 'transitive_closure' is not defined

In [37]:
_ = os.system("say done")

NameError: name 'os' is not defined

In [38]:
linear_arg_adjacency_matrix = linearize_brick_graph(brick_graph_closure)

NameError: name 'linearize_brick_graph' is not defined

In [39]:
%%time
linarg_old_triangular = linarg_old.make_triangular()

NameError: name 'linarg_old' is not defined

In [40]:
v = np.ones(linarg_old_triangular.shape[0])
ac = v @ linarg_old_triangular
ac_true = v @ genotypes
np.all(ac == ac_true)

NameError: name 'np' is not defined

In [41]:
ac, ac_true

NameError: name 'ac' is not defined

In [42]:
%%time
path = "/Users/loconnor/Dropbox/linearARG/linearg_shared/data/1kg_ALL_chr22_47876022_49355984_MAF_0.01_RF_0.01_T_8.genos"
output_file_prefix = None # "/Volumes/T7/data/test/1kg_chr22_47876022_49355984"
linarg_old, genotypes = ld.run_linarg_workflow(
    path, output_file_prefix, flip_minor_alleles=True, recombination_method="old", make_triangular=False, brick_graph_method="new"
)

NameError: name 'ld' is not defined

In [43]:
genotypes

NameError: name 'genotypes' is not defined

In [44]:
%%time
bg = BrickGraphPy.from_genotypes(csc_matrix(genotypes))

NameError: name 'BrickGraphPy' is not defined

In [45]:
_ = os.system("say done")

NameError: name 'os' is not defined

In [46]:
def transitive_closure(A: csr_matrix):
    G = nx.from_scipy_sparse_array(A, create_using=nx.DiGraph())
    for i in G.nodes():
        G.add_edge(i, i)
    G = nx.transitive_closure(G)
    return nx.to_scipy_sparse_array(G, format="csr")

NameError: name 'csr_matrix' is not defined

In [47]:
brick_graph_closure = bg.to_csr()
brick_graph_closure = remove_undirected_edges(brick_graph_closure)
brick_graph_closure.diagonal()

NameError: name 'bg' is not defined

In [48]:
A = transitive_closure(brick_graph_closure)
A.nnz, brick_graph_closure.nnz

NameError: name 'transitive_closure' is not defined

In [49]:
brick_graph_closure_old = infer_brick_graph_using_containment(genotypes, 1)

NameError: name 'infer_brick_graph_using_containment' is not defined

In [50]:
brick_graph_closure, brick_graph_closure_old

NameError: name 'brick_graph_closure' is not defined

In [51]:
count = 0
for i in range(len(brick_graph_closure.indptr) - 1):
    for j in brick_graph_closure.indices[brick_graph_closure.indptr[i]:brick_graph_closure.indptr[i + 1]]:
        if brick_graph_closure_old[i,j] == 0:
            count += 1
            if np.sum(genotypes[:,i] != genotypes[:,j]) > 0:
                print(i,j)
count

NameError: name 'brick_graph_closure' is not defined

In [52]:
brick_graph_closure_old.nnz, brick_graph_closure.nnz

NameError: name 'brick_graph_closure_old' is not defined

In [53]:
linearize_brick_graph(brick_graph_closure_old).nnz


NameError: name 'linearize_brick_graph' is not defined

In [54]:
linear_arg_adjacency_matrix = linearize_brick_graph(brick_graph_closure)
linear_arg_adjacency_matrix.nnz

NameError: name 'linearize_brick_graph' is not defined

In [55]:
linear_arg_adjacency_matrix.diagonal()

NameError: name 'linear_arg_adjacency_matrix' is not defined

In [56]:
linear_arg_adjacency_matrix = add_singleton_variants(genotypes, linear_arg_adjacency_matrix)
linear_arg_adjacency_matrix = add_samples_to_linear_arg(genotypes, linear_arg_adjacency_matrix)

NameError: name 'add_singleton_variants' is not defined

In [57]:
n, m = genotypes.shape
samples_idx = np.arange(n)
variants_idx = np.arange(n, m + n)
flip = np.zeros(len(variants_idx), dtype=bool)

linarg_new = ld.LinearARG(linear_arg_adjacency_matrix, samples_idx, variants_idx, flip)

NameError: name 'genotypes' is not defined

In [58]:
linarg_new.nnz, linarg_old.nnz

NameError: name 'linarg_new' is not defined

In [59]:
linarg_new.A.diagonal(), linarg_old.A.diagonal()

NameError: name 'linarg_new' is not defined

In [60]:
linarg_new_recom = linarg_new.unweight()
linarg_new_recom = linarg_new_recom.find_recombinations()

NameError: name 'linarg_new' is not defined

In [61]:
genotypes.nnz / linarg_new_recom.nnz

NameError: name 'genotypes' is not defined

In [62]:
linarg_old_recom = linarg_old.unweight()
linarg_old_recom = linarg_old_recom.find_recombinations()
genotypes.nnz / linarg_old_recom.nnz

NameError: name 'linarg_old' is not defined

In [63]:
linarg_new_triangular = linarg_new_recom.make_triangular()
linarg_old_triangular = linarg_old_recom.make_triangular()

NameError: name 'linarg_new_recom' is not defined

In [64]:
linarg_old_recom_triangular = linarg_old_recom.make_triangular()
v = np.ones(linarg_new_recom.shape[0])
ac = v @ linarg_old_recom_triangular
ac_true = v @ genotypes
np.all(ac == ac_true)

NameError: name 'linarg_old_recom' is not defined

In [65]:
v = np.ones(linarg_new_recom.shape[0])
ac = v @ linarg_new_triangular
ac_true = v @ genotypes
np.all(ac == ac_true)

NameError: name 'np' is not defined

In [66]:
ac, ac_true

NameError: name 'ac' is not defined

In [67]:
linarg_new.A.diagonal()

NameError: name 'linarg_new' is not defined

In [68]:
linarg_old.A.diagonal()

NameError: name 'linarg_old' is not defined

In [69]:
brick_graph_closure.indices

NameError: name 'brick_graph_closure' is not defined

In [70]:
import networkx as nx
G = nx.DiGraph()
G.add_edge(0,1)
A = nx.to_scipy_sparse_array(G, format="csc", dtype=np.intc)
A

NameError: name 'np' is not defined

In [71]:
A=nx.to_scipy_sparse_array(G, format="csr", dtype=np.intc).transpose()

NameError: name 'np' is not defined

In [72]:
A.toarray()

NameError: name 'A' is not defined

In [73]:
brick_graph_closure_old = infer_brick_graph_using_containment(genotypes, 1)

NameError: name 'infer_brick_graph_using_containment' is not defined

In [74]:
brick_graph_closure_old.nnz, brick_graph_closure.nnz

NameError: name 'brick_graph_closure_old' is not defined

In [75]:
more_than_one_carrier = np.diff(genotypes.indptr) > 1
np.mean(more_than_one_carrier) == 1

NameError: name 'np' is not defined

In [76]:
count = 0
for i in range(len(brick_graph_closure.indptr) - 1):
    for j in brick_graph_closure.indices[brick_graph_closure.indptr[i]:brick_graph_closure.indptr[i + 1]]:
        if brick_graph_closure_old[i,j] == 0:
            count += 1
count

NameError: name 'brick_graph_closure' is not defined

In [77]:
brick_graph_closure = brick_graph_closure.transpose()

NameError: name 'brick_graph_closure' is not defined

In [78]:
brick_graph_closure

NameError: name 'brick_graph_closure' is not defined

In [79]:
np.all(brick_graph_closure.diagonal() == 1)

NameError: name 'np' is not defined

In [80]:
for i, idx in enumerate(brick_graph_closure.indptr[1:]):
    assert brick_graph_closure.indices[idx - 1] == i, f"{i}, {idx}"

NameError: name 'brick_graph_closure' is not defined

In [81]:
brick_graph_closure.nnz

NameError: name 'brick_graph_closure' is not defined

In [82]:
type(brick_graph_closure.indptr)

NameError: name 'brick_graph_closure' is not defined

In [83]:
%%time
linarg_recom = linarg.unweight()
linarg_recom = linarg_recom.find_recombinations()
linarg_recom = linarg_recom.make_triangular()
linarg.nnz, linarg_recom.nnz, genotypes.nnz

NameError: name 'linarg' is not defined

In [84]:
linarg_recom_subset = linarg_recom.__getitem__((slice(100),slice(100)))

NameError: name 'linarg_recom' is not defined

In [85]:
linarg_recom_subset.A.data[]


SyntaxError: invalid syntax (479203741.py, line 1)

In [86]:
linarg_recom_subset.A[10,]

NameError: name 'linarg_recom_subset' is not defined

In [ ]:
linarg_recom_subset = linarg_recom[:100,:]


In [ ]:
a = np.zeros((10,10))
a[:1, ]

In [ ]:
linarg_triangular = linarg.make_triangular()

In [ ]:
v = np.ones(linarg_recom.shape[0])
ac0 = v @ linarg_triangular
ac1 = v @ linarg_recom
ac2 = v @ genotypes
np.all(ac0 == ac1), np.all(ac0 == ac2)

In [ ]:
linarg_recom = linarg.find_recombinations()
genotypes.nnz / linarg_recom.nnz

In [ ]:
linarg_recom = linarg_recom.make_triangular()
v = np.ones(linarg_triangular.shape[0])
ac0 = v @ linarg_recom
ac1 = v @ genotypes
np.all(ac0 == ac1)

In [ ]:
G = np.zeros((5, 5))
G[:3, 3:] = 1
G = csr_matrix(G)

In [ ]:
trio_list = ld.Trios(100)
trio_list.convert_matrix(G.indices, G.indptr, G.shape[0])

In [ ]:
edges = trio_list.fill_edgelist()
print(np.asarray(edges))

In [ ]:
trio_list.find_recombinations()
edges = trio_list.fill_edgelist()
print(np.asarray(edges))

In [ ]:
linarg_recom = linarg.find_recombinations()

In [ ]:
genotypes.nnz / linarg_recom.nnz

In [1]:
linarg.nnz

NameError: name 'linarg' is not defined

In [ ]:
linarg.A.shape, linarg_recom.A.shape

In [ ]:
linarg.variant_indices

In [ ]:
linarg.sample_indices

In [ ]:
linarg.A[:, 4000]

In [ ]:
n = b.get_graph().number_of_nodes
edges = b.get_graph().edge_list()
rows, cols = zip(*edges)

M = csr_matrix((np.ones(len(edges)), (list(rows), list(cols))), shape=(n, n))

In [ ]:
def break_ties(haplotypes):
    ties = haplotypes.multiply(haplotypes.transpose())
    haplotypes = haplotypes - triu(ties, k=1)
    haplotypes.eliminate_zeros()
    return haplotypes

In [ ]:
M = break_ties(M)

In [ ]:
R = inv(csc_matrix(eye(M.shape[0]) - M))

In [ ]:
R.data = np.round(R.data)
R.eliminate_zeros()

In [ ]:
R.data = np.ones_like(R.data)

In [ ]:
A = eye(n) - inv(R)

In [ ]:
A.nnz, R.nnz

In [ ]:
np.mean(A.data < 0)

In [ ]:
np.sum(A.data == -1)

In [ ]:
As = geno * csc_matrix(eye(M.shape[0]) - A)

In [ ]:
As.nnz, geno.nnz

In [ ]:
linarg.nnz, genotypes.nnz

In [2]:
linarg.A[linarg.variant_indices, :][:, linarg.variant_indices].nnz

NameError: name 'linarg' is not defined

In [3]:
A.nnz

NameError: name 'A' is not defined

In [4]:
ones_vector = np.arange(genotypes.shape[0])
allele_count1 = ones_vector @ genotypes
allele_count2 = ones_vector @ linarg
np.all(allele_count1 == allele_count2)

NameError: name 'np' is not defined

In [5]:
(
    np.mean(linarg.A.data == 1),
    np.mean(linarg.A.data == -1),
    np.mean(linarg.A.data == -1),
    np.mean(linarg.A.data < -1),
    np.mean(linarg.A.data > 1),
)

NameError: name 'np' is not defined

In [6]:
np.min(linarg.A.data)

NameError: name 'np' is not defined

In [7]:
np.round(3.0) == 3.0

NameError: name 'np' is not defined

In [8]:
geno = genotypes  # [:1000,:2000]

NameError: name 'genotypes' is not defined

In [9]:
%%time
B = ld.BrickGraph.from_csc(geno.indptr, geno.indices, geno.shape[0])

NameError: name 'ld' is not defined

In [10]:
B.get_tree().number_of_nodes, B.get_tree().number_of_edges

NameError: name 'B' is not defined

In [11]:
B.get_graph().number_of_nodes, B.get_graph().number_of_edges

NameError: name 'B' is not defined

In [12]:
B = from_csc(geno.indptr, geno.indices, geno.shape[0], 10)

NameError: name 'from_csc' is not defined

In [13]:
edges = B.edge_list()

NameError: name 'B' is not defined

In [14]:
import networkx as nx


G = nx.DiGraph()

In [15]:
G.add_edges_from(edges)

NameError: name 'edges' is not defined

In [ ]:
np.unique([val for key, val in dict(G.out_degree()).items()])

In [ ]:
def from_csc(indptr, indices, num_samples, n):
    num_variants = len(indptr) - 1
    total_allele_count = len(indices)
    result = ld.BrickGraph(num_samples, num_variants, total_allele_count)

    # Forwards pass
    for i in range(n):
        carriers = indices[indptr[i] : indptr[i + 1]]
        result.intersect_clades(carriers, i)
        print(result.get_tree().number_of_nodes, result.get_tree().number_of_edges)

    return result.get_tree()

In [16]:
B = from_csc(geno.indptr, geno.indices, geno.shape[0])

NameError: name 'from_csc' is not defined

In [17]:
i

NameError: name 'i' is not defined

In [18]:
ones_vector = np.ones(1000)
allele_count1 = ones_vector @ genotypes[:1000,]
allele_count2 = ones_vector @ linarg[:1000,]
np.all(allele_count1 == allele_count2)

NameError: name 'np' is not defined

In [19]:
import networkx as nx


def search_descendants(G: nx.DiGraph, nodes: list) -> nx.DiGraph:
    subgraph = nx.DiGraph()
    subgraph.add_nodes_from(nodes)
    nodes_to_search = nodes[::-1].copy()
    leftmost_ancestor = {node: node for node in nodes}
    predecessors = {nodes[i]: nodes[i - 1] if i > 0 else None for i in range(len(nodes))}
    while nodes_to_search:
        node = nodes_to_search.pop()
        la = leftmost_ancestor[node]
        parent_predecessor = predecessors[la]
        if parent_predecessor is None:
            continue
        candidate_other_parents = set(nx.descendants(subgraph, parent_predecessor))
        candidate_other_parents.add(parent_predecessor)
        for child in G.successors(node):
            parents_of_child = list(G.predecessors(child))
            if len(parents_of_child) > 2:
                continue
            assert parents_of_child[0] == node or parents_of_child[1] == node
            other_parent = parents_of_child[0] if parents_of_child[1] == node else parents_of_child[1]
            if other_parent in candidate_other_parents:
                nodes_to_search.append(child)
                leftmost_ancestor[child] = leftmost_ancestor[other_parent]
                subgraph.add_node(child)
                subgraph.add_edge(node, child)
                subgraph.add_edge(other_parent, child)

    return subgraph

In [20]:
subgraph = search_descendants(nx.DiGraph(), [])
subgraph.nodes()

NodeView(())

In [21]:
n = 5
largenumber = 100


def child(p1, p2):
    return p1 + largenumber * p2


G = nx.DiGraph()
G.add_edges_from([(i, child(i, i + 1)) for i in range(n - 1)])
G.add_edges_from([(i + 1, child(i, i + 1)) for i in range(n - 1)])
G.add_edges_from([(i, child(i, i + 2)) for i in range(n - 2)])
G.add_edges_from([(i + 2, child(i, i + 2)) for i in range(n - 2)])
G.add_edges_from([(i, child(i, i + 2)) for i in range(n - 2)])
G.add_edges_from([(i + 2, child(i, i + 2)) for i in range(n - 2)])
c = child(0, 2)
G.add_edge(c, child(c, 4))
G.add_edge(4, child(c, 4))

In [22]:
G.edges()

OutEdgeView([(0, 100), (0, 200), (1, 201), (1, 100), (1, 301), (2, 302), (2, 201), (2, 402), (2, 200), (3, 403), (3, 302), (3, 301), (4, 403), (4, 402), (4, 600), (200, 600)])

In [23]:
linarg.A

NameError: name 'linarg' is not defined

In [24]:
G = nx.from_scipy_sparse_array(linarg.A)

NameError: name 'linarg' is not defined